## Q2. 
Work on the red wine dataset: Wine.db
The data of red wine contains four tables: Wine, Winery, Grape, and Rater.

In [2]:
import os
import sys
import pandas as pd
import sqlite3
import my_packages.Sql_table_details

# Paths and directory
current_directory = os.getcwd()
wine = os.path.join(current_directory, "01-data/01_wine_Project", "wine.db")


# function that checks table details
my_packages.Sql_table_details.display_table_details(wine)
#my_packages.Sql_table_details.sample_table(wine,'winery',10)


Table: Grape
+-----+---------+---------+---------+------------+----+
| cid |   name  |   type  | notnull | dflt_value | pk |
+-----+---------+---------+---------+------------+----+
|  0  |    id   | INTEGER |    0    |    None    | 1  |
|  1  | variety |   TEXT  |    0    |    None    | 0  |
+-----+---------+---------+---------+------------+----+

Table: Winery
+-----+----------+---------+---------+------------+----+
| cid |   name   |   type  | notnull | dflt_value | pk |
+-----+----------+---------+---------+------------+----+
|  0  |    id    | INTEGER |    0    |    None    | 1  |
|  1  | country  |   TEXT  |    0    |    None    | 0  |
|  2  | province |   TEXT  |    0    |    None    | 0  |
|  3  |  winery  |   TEXT  |    0    |    None    | 0  |
+-----+----------+---------+---------+------------+----+

Table: Rater
+-----+----------------+---------+---------+------------+----+
| cid |      name      |   type  | notnull | dflt_value | pk |
+-----+----------------+---------+------

## 2A. 
Grapes are the raw material for making red wine. Which country has the most varieties
of grapes, please report the name of the country and the number of types of grapes

In [3]:
GrapeQuery = """
select 
w2.country,
count(distinct g.variety) as[count of variety]
from Wine w1
 join Winery w2 on w1.wid=w2.id
 join Grape g on  w1.gid=g.id

group by
w2.country

order by count(distinct g.variety) desc
"""
Grapes=my_packages.Sql_table_details.run_query_from_db(wine,GrapeQuery)
#print(Grapes)


+------------------------+------------------+
|        country         | count of variety |
+------------------------+------------------+
|           US           |       294        |
|         Italy          |       174        |
|         France         |       160        |
|         Spain          |       139        |
|        Portugal        |       113        |
|       Australia        |        91        |
|         Chile          |        68        |
|       Argentina        |        68        |
|        Austria         |        56        |
|         Israel         |        54        |
|      South Africa      |        53        |
|         Greece         |        45        |
|      New Zealand       |        34        |
|         Canada         |        33        |
|        Germany         |        32        |
|        Slovenia        |        29        |
|        Hungary         |        29        |
|        Romania         |        27        |
|        Bulgaria        |        

## 2B. 
According to the guide published by wine enthusiasts, red wine can be divided into four
classes based on its price: everyday wine, mid-range wine, fine wine, and super fine wine. If the
price of red wine is higher than or equal to $340, it will be classified as super fine wine. Please
report the average price and points of the super fine wines.

In [4]:
finewineQuery = """
select
avg(points), 
avg(price)
From Wine w
where price >=340
order by price desc

"""
FineWine=my_packages.Sql_table_details.run_query_from_db(wine,finewineQuery)
#print(FineWine)

+-------------------+-------------------+
|    avg(points)    |     avg(price)    |
+-------------------+-------------------+
| 95.39755351681957 | 629.1896024464831 |
+-------------------+-------------------+


## 2C.
C. A rater evaluates the score and price of the wine fairly and unbiasedly based on the
quality of the wine. Please write SQL code and python code to analyse the data statistics of points
and prices of wine evaluated by each taster and use the format rater_name.xlsx to export the data
statistics.

In [5]:
RaterQuery = """
SELECT
    r.name AS [Rater],
    w.points , 
    w.price
FROM Wine w
JOIN Rater r ON w.rid = r.id

"""
Raters_table = my_packages.Sql_table_details.run_query_from_db(wine,RaterQuery,print_table=False)

grouped_statistics = Raters_table.groupby('Rater')['points'].describe()
median_series = Raters_table.groupby('Rater')['points'].median()

grouped_statistics = grouped_statistics.reset_index()
grouped_statistics['median'] = median_series.values
grouped_statistics = grouped_statistics[[
'Rater', 
'count', 
'mean', 
'median', 
'std', 
'min',
 '25%', 
'50%', 
'75%',
 'max']]

grouped_statistics = grouped_statistics.sort_values(by='mean', ascending=False)

print(grouped_statistics)

grouped_statistics.to_excel(current_directory+'/02-output/Raters_names.xlsx', index=False)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('')
print('Analysis Raters_names.xlsx, exported to the 02-output directory ')

                 Rater    count       mean  median       std   min   25%  \
2     Anne Krebiehl MW   4623.0  91.107290    91.0  2.528709  80.0  89.0   
12       Layla Schlack    127.0  90.866142    91.0  1.692008  87.0  90.0   
13       Matt Kettmann  11589.0  90.522823    91.0  2.523812  81.0  89.0   
4    Christina Pickard   1356.0  90.011799    90.0  3.091449  82.0  88.0   
16       Mike DeSimone   1292.0  89.857585    90.0  2.498612  82.0  88.0   
22      Virginie Boone  13198.0  89.794817    90.0  3.148811  80.0  88.0   
17        Paul Gregutt  11494.0  89.390813    90.0  2.819592  80.0  87.0   
10       Kerin O’Keefe  14057.0  89.318560    89.0  2.660627  80.0  87.0   
14     Michael Alberty    196.0  89.260204    89.0  2.080101  83.0  88.0   
19    Sean P. Sullivan   7328.0  89.111217    89.0  2.443419  80.0  87.0   
8           Jim Gordon   7681.0  89.002604    89.0  2.835323  80.0  87.0   
18          Roger Voss  29502.0  88.875059    89.0  2.921718  80.0  87.0   
20       Sta